<a href="https://colab.research.google.com/github/CelikAbdullah/deep-learning-notebooks/blob/main/Computer%20Vision/models/Xception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from tensorflow import keras

# Xception

## The entry flow section with the stem component and projection block

### The stem component

In [15]:
def stem(inputs):
  """ Create the stem component of Xception
      inputs : input tensor to neural network
  """

  # a strided 3x3 convolution for dimensionality reduction to reduce feature maps by 75%
  x = keras.layers.Conv2D(filters=32, kernel_size=3, strides=2)(inputs)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.ReLU()(x)

  # a 3x3 convolution for dimensionality expansion to double the number of filters
  x = keras.layers.Conv2D(filters=64, kernel_size=3, strides=1)(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.ReLU()(x)

  return x

### The Projection Block

In [16]:
def projection_block(x, n_filters):
  """ Build a residual block using depthwise separable convolutions
      with projection shortcut.
      x        : input into residual block
      n_filters: number of filters
  """
  # save the input
  shortcut = x

  # a strided 1x1 convolution to double number of filters in identity link to
  # match output of residual block for the add operation (projection shortcut)
  shortcut = keras.layers.Conv2D(filters=n_filters, kernel_size=1, strides=2, padding='same')(shortcut)
  shortcut = keras.layers.BatchNormalization()(shortcut)

  # first depthwise separable convolution
  x = keras.layers.SeparableConv2D(filters=n_filters, kernel_size=3, padding='same')(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.ReLU()(x)

  # second depthwise separable convolution
  x = keras.layers.SeparableConv2D(filters=n_filters, kernel_size=3, padding='same')(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.ReLU()(x)

  # create pooled feature maps, reduce size by 75%
  x = keras.layers.MaxPooling2D(pool_size=3, strides=2, padding='same')(x)

  # add the projection shortcut to the output of the block
  x = keras.layers.Add()([x, shortcut])

  return x

### The Entry Flow

In [17]:
def entryFlow(inputs):
  """ create the entry flow section
      inputs : input tensor to neural network
  """

  # create the stem to the neural network
  x = stem(inputs)

  # create three residual blocks using linear projection
  for n_filters in [128, 256, 728]:
    x = projection_block(x, n_filters)


  return x

## The middle flow section with a residual block

### The Residual Block

In [18]:
def residual_block(x, n_filters):
  """ Create a residual block using depthwise separable convolutions
      x        : input into residual block
      n_filters: number of filters
  """

  # save the input
  shortcut = x

  # a 3x3 depthwise separable convolution
  x = keras.layers.SeparableConv2D(filters=n_filters, kernel_size=3, padding='same')(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.ReLU()(x)

  # a 3x3 depthwise separable convolution
  x = keras.layers.SeparableConv2D(filters=n_filters, kernel_size=3, padding='same')(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.ReLU()(x)

  # a 3x3 depthwise separable convolution
  x = keras.layers.SeparableConv2D(filters=n_filters, kernel_size=3, padding='same')(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.ReLU()(x)

  # we add the identity link to the output of the block
  x = keras.layers.Add()([x, shortcut])
  return x

### The Middle Flow

In [19]:
def middleFlow(x):
  """ Create the middle flow section
      x : input tensor into section
  """
  # we create 8 residual blocks
  for _ in range(8):
    x = residual_block(x, 728)

  return x

## The exit flow section with a task component

### The task component

In [20]:
def task(x, n_classes):
  """ The task component of Xception.
      x         : input to the classifier
      n_classes : number of output classes
  """
  # Global Average Pooling will flatten the 10x10 feature maps into 1D
  # feature maps
  x = keras.layers.GlobalAveragePooling2D()(x)

  # Fully connected output layer (classification)
  x = keras.layers.Dense(units=n_classes, activation='softmax')(x)

  return x

### The Exit Flow

In [21]:
def exitFlow(x, n_classes):
  """ Create the exit flow section
      x         : input to the exit flow section
      n_classes : number of output classes
  """

  # 1x1 strided convolution to increase number and reduce size of feature maps
  # in identity link to match output of residual block for the add operation (projection shortcut)
  shortcut = keras.layers.Conv2D(filters=1024, kernel_size=1, strides=2, padding='same')(x)
  shortcut = keras.layers.BatchNormalization()(shortcut)

  # a 3x3 depthwise separable convolution for dimensionality reduction (reduce number of filters)
  x = keras.layers.SeparableConv2D(filters=728, kernel_size=3, padding='same')(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.ReLU()(x)

  # a 3x3 depthwise separable convolution for dimensionality restoration
  x = keras.layers.SeparableConv2D(filters=1024, kernel_size=3, padding='same')(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.ReLU()(x)

  # a 3x3 max pooling to create pooled feature maps, reduce size by 75%
  x = keras.layers.MaxPooling2D(pool_size=3, strides=2, padding='same')(x)

  # we add the projection shortcut to the output of the pooling layer
  x = keras.layers.Add()([x, shortcut])

  # a 3x3 depthwise separable convolution
  x = keras.layers.SeparableConv2D(filters=1556, kernel_size=3, padding='same')(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.ReLU()(x)

  # a 3x3 depthwise separable convolution
  x = keras.layers.SeparableConv2D(filters=2048, kernel_size=3, padding='same')(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.ReLU()(x)

  # pass 'x' to the task component for classification
  x = task(x, n_classes)

  return x

## The Xception model

In [22]:
def build_xception(shape=(299,299,3), classes=1000):
  # create the input vector
  inputs = keras.Input(shape=(299, 299, 3))

  # pass the inputs through the entry flow section
  x = entryFlow(inputs)

  # pass output of entry flow through the middle flow section
  x = middleFlow(x)

  # pass output of middle flow through the exit flow section
  outputs = exitFlow(x, classes)

  # Instantiate the model
  return keras.Model(inputs=inputs, outputs=outputs, name="Xception")


In [23]:
# create the Xception model
xception_model = build_xception()
# print the xception model
xception_model.summary()

Model: "Xception"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 conv2d_6 (Conv2D)           (None, 149, 149, 32)         896       ['input_2[0][0]']             
                                                                                                  
 batch_normalization_40 (Ba  (None, 149, 149, 32)         128       ['conv2d_6[0][0]']            
 tchNormalization)                                                                                
                                                                                                  
 re_lu_36 (ReLU)             (None, 149, 149, 32)         0         ['batch_normalization_4